# Evaluating Whisper
In this notebook we evaluate Whisper. The notebook assumes that test data is uploaded to an S3 bucket and a endpoint has been created.

In [ ]:
# !pip install evaluate

In [ ]:
# !pip install jiwer

In [1]:
import sagemaker
import boto3

sess = sagemaker.Session()
sagemaker_session_bucket = sess.default_bucket()

Getting the list of audio files in the S3 bucket which we will use for testing

In [30]:
s3 = boto3.client('s3')
file_list = []

response = s3.list_objects_v2(
    Bucket=sagemaker_session_bucket,
    Prefix='whisper/data/test/he/')

for content in response.get('Contents', []):
    if "wav" in content['Key']:
        file_list.append(f"s3://{sagemaker_session_bucket}/{content['Key']}")

Downloading the csv file that contains the ground truth

In [43]:
s3.download_file(sagemaker_session_bucket, "whisper/data/test/he/test-he.tsv", 'test-he.tsv')

In [52]:
import pandas as pd
df = pd.read_csv('test-he.tsv', sep='\t')
df["transcription_whisper"] = ""

In [ ]:
df.head()

Loading the endpoint

In [11]:
from sagemaker.huggingface import HuggingFacePredictor

endpoint_name = "whisper-large-custom-2022-12-08-21-23-42-385"
predictor = HuggingFacePredictor(endpoint_name)

Loop over the file list - call the endpoint for each file.

Todo: This can be improved in two ways:
1. Parallel calls to the endpoint (concurrency)
2. Using Batch Transform instead (tbc if this is possible)

In [74]:
transcription_list = []

for i, file in enumerate(file_list):
    print(i)
    
    data = {
        "s3_file": file,
        "language": "he"
    }
    res = predictor.predict(data=data)
    transcription = res['transcription']
    
    df.at[i, "transcription_whisper"] = transcription

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [75]:
df.head()

,id,raw_transcription,transcription,new_filename,transcription_whisper
0,1741,אזורים גדולים צפונית משם הם בעלי אוכלוסייה דלי...,אזורים גדולים צפונית משם הם בעלי אוכלוסייה דלי...,test-he-000.wav,אזורים גדולים צפונית משם הם בעלי אוכלוסייה די...
1,1861,"כתוצאה מכך, השחקנים מעשנים ג'וינטים של קנאביס ...",כתוצאה מכך השחקנים מעשנים ג'וינטים של קנאביס ע...,test-he-001.wav,כתוצאה מכך השחקנים מאשנים ג'וינטים של קנאביס ...
2,1803,"הלחות על הידיים שלך תגיב עם השכבות החיצוניות, ...",הלחות על הידיים שלך תגיב עם השכבות החיצוניות מ...,test-he-002.wav,"הלחות על הידיים שלך תגיב עם משחבות החיצוניות,..."
3,1873,"היו תקיפות בדחיית גברים, ואל תפחדו לעמוד על של...",היו תקיפות בדחיית גברים ואל תפחדו לעמוד על שלכ...,test-he-003.wav,לא תקיפו על ביטחיית גברים ואל תפחדו ללמוד לדש...
4,1759,"ד""ר מלאר בלסוברמניאן, בן 29, נמצא בבלו אש, אוה...","ד""ר מלאר בלסוברמניאן בן 29 נמצא בבלו אש אוהיו ...",test-he-004.wav,"ד.מ.ב.מ.ע.ב.29 נמצא בבלו אש, הוהיו. פרוור נמצ..."


## Overall WER score

In [108]:
predictions = list(df["transcription_whisper"])
references = list(df["raw_transcription"])

In [109]:
import evaluate
wer = evaluate.load("wer")

In [110]:
wer_score = wer.compute(predictions=predictions, references=references)
wer_score

0.3350485991995426

## Individual WER Score

In [91]:
def compute_wer(row, col1, col2):
    return wer.compute(predictions=[row[col1]], references=[row[col2]])

In [92]:
df.head()

,id,raw_transcription,transcription,new_filename,transcription_whisper
0,1741,אזורים גדולים צפונית משם הם בעלי אוכלוסייה דלי...,אזורים גדולים צפונית משם הם בעלי אוכלוסייה דלי...,test-he-000.wav,אזורים גדולים צפונית משם הם בעלי אוכלוסייה די...
1,1861,"כתוצאה מכך, השחקנים מעשנים ג'וינטים של קנאביס ...",כתוצאה מכך השחקנים מעשנים ג'וינטים של קנאביס ע...,test-he-001.wav,כתוצאה מכך השחקנים מאשנים ג'וינטים של קנאביס ...
2,1803,"הלחות על הידיים שלך תגיב עם השכבות החיצוניות, ...",הלחות על הידיים שלך תגיב עם השכבות החיצוניות מ...,test-he-002.wav,"הלחות על הידיים שלך תגיב עם משחבות החיצוניות,..."
3,1873,"היו תקיפות בדחיית גברים, ואל תפחדו לעמוד על של...",היו תקיפות בדחיית גברים ואל תפחדו לעמוד על שלכ...,test-he-003.wav,לא תקיפו על ביטחיית גברים ואל תפחדו ללמוד לדש...
4,1759,"ד""ר מלאר בלסוברמניאן, בן 29, נמצא בבלו אש, אוה...","ד""ר מלאר בלסוברמניאן בן 29 נמצא בבלו אש אוהיו ...",test-he-004.wav,"ד.מ.ב.מ.ע.ב.29 נמצא בבלו אש, הוהיו. פרוור נמצ..."


In [93]:
df["WER score - Whisper"] = df.apply(compute_wer, args=('transcription_whisper', 'raw_transcription'), axis=1)

In [94]:
df.head()

,id,raw_transcription,transcription,new_filename,transcription_whisper,WER score - Whisper
0,1741,אזורים גדולים צפונית משם הם בעלי אוכלוסייה דלי...,אזורים גדולים צפונית משם הם בעלי אוכלוסייה דלי...,test-he-000.wav,אזורים גדולים צפונית משם הם בעלי אוכלוסייה די...,0.187500
1,1861,"כתוצאה מכך, השחקנים מעשנים ג'וינטים של קנאביס ...",כתוצאה מכך השחקנים מעשנים ג'וינטים של קנאביס ע...,test-he-001.wav,כתוצאה מכך השחקנים מאשנים ג'וינטים של קנאביס ...,0.200000
2,1803,"הלחות על הידיים שלך תגיב עם השכבות החיצוניות, ...",הלחות על הידיים שלך תגיב עם השכבות החיצוניות מ...,test-he-002.wav,"הלחות על הידיים שלך תגיב עם משחבות החיצוניות,...",0.133333
3,1873,"היו תקיפות בדחיית גברים, ואל תפחדו לעמוד על של...",היו תקיפות בדחיית גברים ואל תפחדו לעמוד על שלכ...,test-he-003.wav,לא תקיפו על ביטחיית גברים ואל תפחדו ללמוד לדש...,0.578947
4,1759,"ד""ר מלאר בלסוברמניאן, בן 29, נמצא בבלו אש, אוה...","ד""ר מלאר בלסוברמניאן בן 29 נמצא בבלו אש אוהיו ...",test-he-004.wav,"ד.מ.ב.מ.ע.ב.29 נמצא בבלו אש, הוהיו. פרוור נמצ...",0.322581


Sort descending by WER score to see where the model struggled the most:

In [95]:
df.sort_values("WER score - Whisper", ascending=False).head(10)

,id,raw_transcription,transcription,new_filename,transcription_whisper,WER score - Whisper
18,1850,"זה יאפשר לו להיות תואם לאחור עם 802.11a, 802.1...",זה יאפשר לו להיות תואם לאחור עם 802.11a 802.11...,test-he-018.wav,בתנאי שלתחנת הבסיס יש מכשירי קשר כפולים 802.1...,1.000000
36,1885,בין השעות 22:00-23:00 לפי אזור זמן ההרים (MDT)...,בין השעות 22:00-23:00 לפי אזור זמן ההרים mdt ה...,test-he-036.wav,פני הים,1.000000
80,1870,הסצנות מוצגות על גבי הפירמידות והפירמידות השונ...,הסצנות מוצגות על גבי הפירמידות והפירמידות השונ...,test-he-080.wav,הצנות נוצרות אגב הפרמילו והפרמילות השונות מרוב.,0.875000
73,1672,היזהרו שהבד לא יתחמם יותר מידי (מה שיכול לגרום...,היזהרו שהבד לא יתחמם יותר מידי מה שיכול לגרום ...,test-he-073.wav,זהרו שהבד לא יתחנה מיותר מדי.,0.857143
68,1731,בארכיפלג האיים ובאגמים אינכם צריכים בהכרח יאכטה.,בארכיפלג האיים ובאגמים אינכם צריכים בהכרח יאכטה,test-he-068.wav,"באר חבלק, העיים ובעגמים אינכם צריכים בהכריח י...",0.857143
65,1668,"ייתכן גם שכדאי לרכוש Wild Card, המאפשר כניסה ל...",ייתכן גם שכדאי לרכוש wild card המאפשר כניסה למ...,test-he-065.wav,המאפשר כניסה למבחרת פארקים בדרום אפריקה או לכ...,0.777778
39,1918,יער סונדרבאנס הוכרז כאתר מורשת עולמי מטעם יונס...,יער סונדרבאנס הוכרז כאתר מורשת עולמי מטעם יונס...,test-he-039.wav,ירוק סונדרבנס הוא חצרי אתר מורישת עולמים מטעם...,0.764706
53,1940,"בהתחשב בכל הנתונים, אל לנו להיות מופתעים אם אב...",בהתחשב בכל הנתונים אל לנו להיות מופתעים אם אבו...,test-he-053.wav,"מתחשב בכל הנתונים, אלה נדיות מופתעים עם אהבות...",0.695652
16,1876,מלשכת ניהול מצבי חירום באיי מריאנה הצפוניים נמ...,מלשכת ניהול מצבי חירום באיי מריאנה הצפוניים נמ...,test-he-016.wav,מנשקת מעול מצבי חירום בים אריאן הצפוניים נמסר...,0.692308
66,1725,"ASUS Eee PC, שהושק מוקדם יותר ברחבי העולם מפטת...",asus eee pc שהושק מוקדם יותר ברחבי העולם מפטת ...,test-he-066.wav,יותר בחוויה עולם מפתעת סיבות הקשורות לחסכון ב...,0.681818


In [103]:
df[df["id"] == 1850]['transcription_whisper'].values[0]

' בתנאי שלתחנת הבסיס יש מכשירי קשר כפולים 802.11b 802.11a זה יאפשר לו להיות תואם לאחור עם 802.11'

In [104]:
df[df["id"] == 1850]['raw_transcription'].values[0]

'זה יאפשר לו להיות תואם לאחור עם 802.11a, 802.11b ו-802.11g, בתנאי שלתחנת הבסיס יש מכשירי קשר כפולים.'

## Further processing before calculating WER score
If it makes sense in our use case to do some further processing before calcualting the WER score, we can do that. In this example we remove punctuation before calculating the WER score.

In [105]:
import re

def clean(text):
    return re.sub(r'[^\w\s]', '', text)

In [106]:
predictions_clean = list(map(clean, predictions))
references_clean = list(map(clean, references))

In [107]:
wer_score = wer.compute(predictions=predictions_clean, references=references_clean)
wer_score

0.28219805380652546